In [28]:
import challenge1000
import pandas

In [29]:
raw = pandas.read_csv(challenge1000.RefEnv().transformed.format('df1_clean.csv')).set_index('key_main')
raw = raw.drop('Unnamed: 0', axis = 1)

In [30]:
sch = challenge1000.RefEnv().get_schema()

/Volumes/GoogleDrive/My Drive/Open Data Platform/DATA/SCHEMA/star_schema.json


In [31]:
sch.keys()

dict_keys(['labels', 'template', 'team', 'url_descriptions_formal', 'descriptions_raw', 'descriptions_traduction', 'finance', 'structure', 'people', 'digital_ponctual'])

In [32]:
var = []
for element in sch:
    var = var + [[element, e] for e in sch[element]]

var_map = pandas.DataFrame(var, columns =['label', 'variable'])
var_map['index'] = var_map['label']+'_'+var_map['variable']
var_map = var_map.set_index('index')

var_map

,label,variable
index,,
labels_categories,labels,categories
template_temp,template,temp
team_empl_h_struc,team,empl_h_struc
team_empl_f_struc,team,empl_f_struc
url_descriptions_formal_prez_doc_struc,url_descriptions_formal,prez_doc_struc
...,...,...
digital_ponctual_updated_by_digitafr,digital_ponctual,updated_by_digitafr
digital_ponctual_draft_digitafr,digital_ponctual,draft_digitafr
digital_ponctual_ip_digitafr,digital_ponctual,ip_digitafr


In [33]:
header = [e for e in sch['digital_ponctual'] if e in raw.keys()]
100 - raw[header].isna().sum()*100/len(raw)

reco_digitafr            100.000000
cgu_digitafr             100.000000
nwsl_digitafr            100.000000
compte_label_digitafr     97.999420
compte_digitafr           97.999420
mdp_digitafr               0.000000
iduser_digitafr           16.439548
time_digitafr            100.000000
last_update_digitafr     100.000000
created_by_digitafr       16.439548
updated_by_digitafr       16.439548
draft_digitafr           100.000000
ip_digitafr              100.000000
id_digitafr              100.000000
key_digitafr             100.000000
dtype: float64

In [34]:
def completion_rate(type_file):
    h = [e for e in sch[type_file] if e in raw.keys()]
    target = raw[h]
    rate = 100-target.isna().sum()*100/len(target)
    rate = pandas.DataFrame(rate, columns =['completion rate'])
    rate['file'] = type_file
    rate = rate.reset_index()
    rate['variable'] = rate['index']
    rate['index'] = rate['file']+'_'+rate['variable']
    rate = rate[['completion rate', 'file', 'index']].set_index('index')
    return rate

def type_variable(type_file):
    h = [e for e in sch[type_file] if e in raw.keys()]
    target = raw[h]
    header = [['index', 'file', 'variable', 'type', 'nombre de type var']]
    for e in target:
        t_var = list({str(type(a)) for a in set(target[e])})
        nbr_t = len(t_var)
        t_var = '/'.join(t_var)
        header = header +[[type_file +'_'+e, type_file, e,t_var,nbr_t]]
    target = pandas.DataFrame(header[1:], columns = header[0]).set_index('index')
    target = target.drop(['variable','file'], axis =1 )
    return target

def describe_data(type_file):
    h = [e for e in sch[type_file] if e in raw.keys()]
    try:
        target = raw[h].describe().transpose().reset_index()
        target['variable'] = target['index']
        target['index'] = type_file + '_' + target['index']
        target = target.set_index('index')
        target = target.drop('variable', axis =1 )
        return target
    except:
        pass


In [46]:
df_target = var_map.join(pandas.concat([completion_rate(e) for e in sch], axis = 0))
df_target = df_target.join(pandas.concat([describe_data(e) for e in sch], axis = 0))
df_target = df_target.join(pandas.concat([type_variable(type_file)for type_file in set(sch)], axis = 0))
df_target['platform'] = 'Challenge 1000'
for element in ['completion rate', 'count', 'mean', 'std','min', '25%', '50%', '75%', 'max', 'unique', 'freq','nombre de type var']:
    df_target[element] = df_target[element].map(float)
df_target.fillna('', inplace = True)

In [47]:
from DA_IO import Gspreadsheet
worksheet_name = 'audit data'
gsp = Gspreadsheet(worksheet_name)
#gsp.create()
sheet_name = 'challenge 1000'
#gsp.add_sheet(sheet_name)
gsp.df_to_sheet(df = df_target, sheet_name = sheet_name)

https://docs.google.com/spreadsheets/d/1f4nrE397_sFVEPAuUYnUAdwmBs_HTMh56dnthT3w84A/edit#gid=0
